In [ ]:
import wandb
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Whisker
from bokeh.transform import factor_cmap, jitter
from bokeh.layouts import gridplot

from fundusClassif.data.data_factory import get_datamodule_from_config
from nntools.utils import Config


In [ ]:
api = wandb.Api()
runs = api.runs("hmr24/FundusDRGrading-src_fundusClassif_scripts")

important_keys = ['lr','preprocessing', 'data_augmentation_type']
list_configs = []
for i, r in enumerate(runs):
    if i < 74:
        pass
    else :
        kappa = r.summary_metrics.get("Validation Quadratic Kappa", None)
        kappa_ddr = r.summary_metrics.get("Quadratic Kappa_DDR_test", None)
        kappa_eyepacs = r.summary_metrics.get("Quadratic Kappa_EYEPACS_test", None)
        kappa_idrid = r.summary_metrics.get("Quadratic Kappa_IDRID_test", None)
        if kappa is None:
            continue
        config = {k: v for k, v in r.config.items() if k in important_keys}
        config['name'] = r.name
        config['kappa'] = kappa
        config['kappa_ddr'] = kappa_ddr
        config['kappa_eyepacs'] = kappa_eyepacs
        config['kappa_idrid'] = kappa_idrid
    
        list_configs.append(config)

df = pd.DataFrame(list_configs)
df

In [ ]:
df_preprocessing_type = df[df['preprocessing'] == 'sarki']

config = Config('../configs/config.yaml')
datamodule = get_datamodule_from_config(config['datasets'], config['data'])
test_dataloader = datamodule.test_dataloader()
test_datasets_ids = [d.dataset.id for i, d in enumerate(test_dataloader)]
dataloader0_weight = 0
dataloader1_weight = 0
dataloader2_weight = 0
lenght_total_test = 0
for i, d in enumerate(test_dataloader):
    lenght_total_test += len(d.dataset)
#print(lenght_total_test)

for i, d in enumerate(test_dataloader):
    if d.dataset.id == test_datasets_ids[0]:
        dataloader0_weight = len(d.dataset)/lenght_total_test
        #print(dataloader0_weight)
        #print(d.dataset.id)
    elif d.dataset.id == test_datasets_ids[1]:
        dataloader1_weight += len(d.dataset)/lenght_total_test
        #print(dataloader1_weight)
        #print(d.dataset.id)
    elif d.dataset.id == test_datasets_ids[2]:
        dataloader2_weight += len(d.dataset)/lenght_total_test
        #print(dataloader2_weight)
        #print(d.dataset.id)

df_preprocessing_type['kappa_weighted'] = 0 
for i in range(len(df_preprocessing_type)):
    eyepacs = df_preprocessing_type['kappa_eyepacs'].iloc[i]*dataloader0_weight
    idrid = df_preprocessing_type['kappa_idrid'].iloc[i]*dataloader1_weight
    ddr = df_preprocessing_type['kappa_ddr'].iloc[i]*dataloader2_weight
    df_preprocessing_type['kappa_weighted'].iloc[i] =  eyepacs + idrid + ddr
df_preprocessing_type

